In [16]:
import os
import trimesh
import numpy as np
import pandas as pd
from itertools import combinations
from trimesh.proximity import ProximityQuery

# === 工具函数 ===

def save_mesh_to_csv(mesh, base_name, folder):
    """将 mesh 保存为两个 CSV 文件"""
    vertices_df = pd.DataFrame(mesh.vertices, columns=['x', 'y', 'z'])
    faces_df = pd.DataFrame(mesh.faces, columns=['v0', 'v1', 'v2'])
    vertices_df.to_csv(os.path.join(folder, f"{base_name}_vertices.csv"), index_label='id')
    faces_df.to_csv(os.path.join(folder, f"{base_name}_faces.csv"), index_label='id')

def load_mesh_from_csv(base_name, folder):
    """从两个 CSV 文件重建 mesh"""
    v_path = os.path.join(folder, f"{base_name}_vertices.csv")
    f_path = os.path.join(folder, f"{base_name}_faces.csv")
    verts = pd.read_csv(v_path, index_col=0)[['x', 'y', 'z']].values
    faces = pd.read_csv(f_path, index_col=0)[['v0', 'v1', 'v2']].values
    return trimesh.Trimesh(vertices=verts, faces=faces, process=True)

def CollisionFree(mesh_A, mesh_B, name_A="", name_B="", tol=1e-6):
    try:
        inter = mesh_A.intersection(mesh_B, engine='manifold')
        if not inter.is_empty and inter.volume > tol:
            volume = inter.volume
            print(f"❌ {name_A} 与 {name_B} 干涉！交集体积 = {volume:.6f}")
            return (False, volume)
        else:
            pq_A = ProximityQuery(mesh_A)
            pq_B = ProximityQuery(mesh_B)
            d1 = pq_B.signed_distance(mesh_A.vertices)
            d2 = pq_A.signed_distance(mesh_B.vertices)
            min_dist = np.min(np.abs(np.concatenate([d1, d2])))
            print(f"✅ {name_A} 与 {name_B} 无干涉，最小距离 = {min_dist:.6f}")
            return (True, min_dist)
    except Exception as e:
        print(f"⚠️ {name_A} 与 {name_B} 检测失败：{e}")
        return (None, None)

# === 处理主逻辑 ===

model_dir = 'model'
file_list = [f for f in os.listdir(model_dir) if f.endswith('.stl')]

# 先把每个 STL 存为 CSV，然后从 CSV 加载回 mesh
meshes = {}
for fname in file_list:
    name = os.path.splitext(fname)[0]
    path = os.path.join(model_dir, fname)
    mesh = trimesh.load(path, force='mesh')

    # 导出为 CSV
    save_mesh_to_csv(mesh, name, model_dir)

    # 从 CSV 重建
    mesh_from_csv = load_mesh_from_csv(name, model_dir)
    meshes[fname] = mesh_from_csv

print(f"✅ 共处理 {len(meshes)} 个 STL 模型")

# === 遍历组合进行干涉检测 ===
results = []
for (name_A, mesh_A), (name_B, mesh_B) in combinations(meshes.items(), 2):
    result = CollisionFree(mesh_A, mesh_B, name_A, name_B)
    results.append((name_A, name_B, *result))

# === 可视化 ===
scene = trimesh.Scene()
for name, mesh in meshes.items():
    mesh.visual.face_colors = [np.random.randint(0, 255) for _ in range(3)] + [150]
    scene.add_geometry(mesh, node_name=name)
scene.show()


✅ 共处理 4 个 STL 模型
❌ body.stl 与 link1.stl 干涉！交集体积 = 89509.252079
❌ body.stl 与 link2.stl 干涉！交集体积 = 85682.426071
❌ body.stl 与 link3.stl 干涉！交集体积 = 116683.415879
❌ link1.stl 与 link2.stl 干涉！交集体积 = 89681.921485
❌ link1.stl 与 link3.stl 干涉！交集体积 = 124601.604949
❌ link2.stl 与 link3.stl 干涉！交集体积 = 126520.013609


In [17]:
from math import radians, cos, sin
from trimesh.transformations import concatenate_matrices

# === STEP 1: 构造 T(l1 -> l2) ===
theta_1 = radians(30)
R12 = np.column_stack([
    [cos(theta_1), 0, sin(theta_1)],
    [sin(theta_1), 0, -cos(theta_1)],
    [0, 1, 0]
])

# R12 = np.column_stack([
#     [cos(theta_1), sin(theta_1), 0],
#     [0, 0, 1],
#     [sin(theta_1), -cos(theta_1), 0]
# ])

print(R12)
p12 = np.array([48, 0, 0])
T12 = np.eye(4)
T12[:3, :3] = R12
T12[:3, 3] = p12

# === STEP 2: 构造 T(l2 -> l3) ===
R23 = np.column_stack([
    [0, 1, 0],
    [-1,  0, 0],
    [0,  0, 1]
])
p23 = np.array([90.16, 0, 0])
T23 = np.eye(4)
T23[:3, :3] = R23
T23[:3, 3] = p23

# === STEP 3: 引用已有的三段 mesh ===
link1 = meshes['link1.stl']
link2 = meshes['link2.stl'].copy()
link3 = meshes['link3.stl'].copy()

# === STEP 4: 应用位姿变换 ===
link2.apply_transform(T12)
link3.apply_transform(T12 @ T23)

# === STEP 5: 颜色区分并添加进场景可视化 ===
link1.visual.face_colors = [0, 150, 255, 180]   # 蓝
link2.visual.face_colors = [0, 255, 100, 180]   # 绿
link3.visual.face_colors = [255, 0, 0, 180]     # 红

scene_leg = trimesh.Scene([link1, link2, link3])
scene_leg.show()


[[ 0.8660254  0.5        0.       ]
 [ 0.         0.         1.       ]
 [ 0.5       -0.8660254  0.       ]]


In [18]:
import numpy as np

def build_T_static():
    """Construct the dictionary of fixed transformation matrices T[(from, to)]"""
    T = {}
    gamma = np.pi / 4  # 45°
    theta_1 = np.radians(30)
    
    x_1 = 79.549
    y_1 = 113.636
    z_1 = 27.5
    
    x_2 = 121.0

    # === Body to leg mount frames (b -> b_i) ===
    def R_z(theta):
        return np.column_stack([
            [np.cos(theta), np.sin(theta), 0],
            [-np.sin(theta),  np.cos(theta), 0],
            [0, 0, 1]
        ])

    # Each leg mount
    T[('b', 'b_1')] = np.eye(4)
    T[('b', 'b_1')][:3, :3] = R_z(gamma)
    # display(T[('b', 'b_1')])
    T[('b', 'b_1')][:3, 3] = [x_1, y_1, z_1]

    T[('b', 'b_2')] = np.eye(4)
    T[('b', 'b_2')][:3, :3] = np.eye(3)
    T[('b', 'b_2')][:3, 3] = [x_2, 0, z_1]

    T[('b', 'b_3')] = np.eye(4)
    T[('b', 'b_3')][:3, :3] = R_z(-gamma)
    T[('b', 'b_3')][:3, 3] = [x_1, -y_1, z_1]

    T[('b', 'b_4')] = np.eye(4)
    T[('b', 'b_4')][:3, :3] = R_z(-gamma + np.pi)
    T[('b', 'b_4')][:3, 3] = [-x_1, y_1, z_1]

    T[('b', 'b_5')] = np.eye(4)
    T[('b', 'b_5')][:3, :3] = R_z(np.pi)
    T[('b', 'b_5')][:3, 3] = [-x_2, 0, z_1]

    T[('b', 'b_6')] = np.eye(4)
    T[('b', 'b_6')][:3, :3] = R_z(gamma + np.pi)
    T[('b', 'b_6')][:3, 3] = [-x_1, -y_1, z_1]

    # === Local transforms along each leg ===
    # T(b_i, l_i_1) = Identity
    for i in range(1, 7):
        T[(f'b_{i}', f'l_{i}_1')] = np.eye(4)

    # T(l_1, l_2) is the same for all legs
    R12 = np.column_stack([
        [np.cos(theta_1), 0, np.sin(theta_1)],
        [np.sin(theta_1), 0, -np.cos(theta_1)],
        [0, 1, 0]
    ])
    T12 = np.eye(4)
    T12[:3, :3] = R12
    T12[:3, 3] = [48, 0, 0]

    # T(l_2, l_3) is also common
    R23 = np.column_stack([
        [0, 1, 0],
        [-1, 0, 0],
        [0, 0, 1]
    ])
    T23 = np.eye(4)
    T23[:3, :3] = R23
    T23[:3, 3] = [90.16, 0, 0]

    for i in range(1, 7):
        T[(f'l_{i}_1', f'l_{i}_2')] = T12.copy()
        T[(f'l_{i}_2', f'l_{i}_3')] = T23.copy()

    # === End-effector transform T(l_i3, e_i) 共用 ===
    R3e = np.column_stack([
        [np.cos(theta_1), np.sin(theta_1), 0],
        [-np.sin(theta_1),  np.cos(theta_1), 0],
        [0,                0,               1]
    ])
    T3e = np.eye(4)
    T3e[:3, :3] = R3e
    T3e[:3, 3] = [146.720, 52.954, 0]

    for i in range(1, 7):
        # l_i_3  →  e_i
        T[(f'l_{i}_3', f'e_{i}')] = T3e.copy()

    return T

# Run to preview dictionary keys
T_static = build_T_static()
list(T_static.keys())


[('b', 'b_1'),
 ('b', 'b_2'),
 ('b', 'b_3'),
 ('b', 'b_4'),
 ('b', 'b_5'),
 ('b', 'b_6'),
 ('b_1', 'l_1_1'),
 ('b_2', 'l_2_1'),
 ('b_3', 'l_3_1'),
 ('b_4', 'l_4_1'),
 ('b_5', 'l_5_1'),
 ('b_6', 'l_6_1'),
 ('l_1_1', 'l_1_2'),
 ('l_1_2', 'l_1_3'),
 ('l_2_1', 'l_2_2'),
 ('l_2_2', 'l_2_3'),
 ('l_3_1', 'l_3_2'),
 ('l_3_2', 'l_3_3'),
 ('l_4_1', 'l_4_2'),
 ('l_4_2', 'l_4_3'),
 ('l_5_1', 'l_5_2'),
 ('l_5_2', 'l_5_3'),
 ('l_6_1', 'l_6_2'),
 ('l_6_2', 'l_6_3'),
 ('l_1_3', 'e_1'),
 ('l_2_3', 'e_2'),
 ('l_3_3', 'e_3'),
 ('l_4_3', 'e_4'),
 ('l_5_3', 'e_5'),
 ('l_6_3', 'e_6')]

In [19]:
from scipy.linalg import expm

def skew(w):
    """Skew-symmetric matrix for a 3D vector"""
    return np.array([
        [0, -w[2], w[1]],
        [w[2], 0, -w[0]],
        [-w[1], w[0], 0]
    ])


def adjoint(T):
    """Compute the 6x6 adjoint representation of transformation matrix T"""
    R = T[:3, :3]
    p = T[:3, 3]
    p_hat = skew(p)
    upper = np.hstack((R, np.zeros((3, 3))))
    lower = np.hstack((p_hat @ R, R))
    return np.vstack((upper, lower))



def compute_screw_axes(T):
    """Compute screw axes B[(i,j)] in body frame using adjoint maps"""
    B = {}
    xi = np.array([0, 0, 1, 0, 0, 0])  # unit screw axis

    for i in range(1, 7):
        # B_i_1 = Ad(T(b, b_i)) * xi
        T1 = T[('b', f'b_{i}')] @ T[(f'b_{i}', f'l_{i}_1')]
        B[(i, 1)] = adjoint(T1) @ xi

        # B_i_2 = Ad(T(b, b_i) * T(b_i, l_i_1) * T(l_i_1, l_i_2)) * xi
        T2 = T1 @ T[(f'l_{i}_1', f'l_{i}_2')]
        B[(i, 2)] = adjoint(T2) @ xi

        # B_i_3 = Ad(T(b, b_i) * T(b_i, l_i_1) * T(l_{i}_1, l_{i}_2) * T(l_{i}_2, l_{i}_3)) * xi
        T3 = T2 @ T[(f'l_{i}_2', f'l_{i}_3')]
        B[(i, 3)] = adjoint(T3) @ xi

    return B

# Test compute
T_static = build_T_static()
B_axes = compute_screw_axes(T_static)

# Show example screw axis B[(1,1)]
B_axes[(1, 2)]


array([ -0.70710678,   0.70710678,   0.        , -19.44543648,
       -19.44543648, 184.60242352])

In [20]:
import trimesh
import numpy as np
import os
from scipy.linalg import expm

import trimesh, numpy as np
from trimesh.transformations import rotation_matrix


# === 工具函数 ===

# =========================================
# 独立的坐标系网格生成函数
# =========================================


def _single_arrow(axis_vec, color_rgba,
                  total_len=40.0, head_frac=0.4,
                  shaft_r=1.2, head_r=4.4):
    """
    生成一根沿 axis_vec 的彩色箭头（圆柱 + 圆锥），
    确保二者端面贴合不出现缝隙。
    """
    import trimesh, numpy as np
    from trimesh.transformations import rotation_matrix

    shaft_len = total_len * (1.0 - head_frac)
    head_len  = total_len * head_frac

    # 1) 圆柱：几何中心在原点 ⇒ 先整体上移 shaft_len/2
    shaft = trimesh.creation.cylinder(
        radius=shaft_r,
        height=shaft_len,
        sections=16)
    shaft.apply_translation([0, 0, shaft_len / 2.0])

    # 2) 圆锥：底面在 z=0，顶在 z=head_len ⇒ 再整体上移 shaft_len
    head = trimesh.creation.cone(
        radius=head_r,
        height=head_len,
        sections=16)
    head.apply_translation([0, 0, shaft_len])

    # 3) 合并并上色
    arrow = trimesh.util.concatenate([shaft, head])
    arrow.visual.face_colors = color_rgba

    # 4) 若 axis_vec ≠ +Z，将箭头旋转到目标方向
    z_axis = np.array([0, 0, 1.0])
    tgt    = np.asarray(axis_vec, dtype=float)
    tgt   /= np.linalg.norm(tgt)
    if not np.allclose(tgt, z_axis):
        rot_axis = np.cross(z_axis, tgt)
        rot_ang  = np.arccos(np.clip(np.dot(z_axis, tgt), -1.0, 1.0))
        arrow.apply_transform(rotation_matrix(rot_ang, rot_axis))

    return arrow



def make_axis_mesh(total_len=40.0):
    """
    返回三色 XYZ 坐标系网格 (Trimesh)：
      X→红，Y→绿，Z→蓝；带箭头。
    """
    arrow_x = _single_arrow([1, 0, 0], [255,   0,   0, 255], total_len)
    arrow_y = _single_arrow([0, 1, 0], [  0, 255,   0, 255], total_len)
    arrow_z = _single_arrow([0, 0, 1], [  0,   0, 255, 255], total_len)
    return trimesh.util.concatenate([arrow_x, arrow_y, arrow_z])

def skew(w):
    return np.array([
        [0, -w[2], w[1]],
        [w[2], 0, -w[0]],
        [-w[1], w[0], 0]
    ])

def hat(B):
    omega = B[:3]
    v = B[3:]
    omega_hat = skew(omega)
    upper = np.hstack((omega_hat, v.reshape(3, 1)))
    return np.vstack((upper, np.array([[0, 0, 0, 0]])))

# === build_hexapod 函数 ===
def build_hexapod(alpha, T, B, meshes):
    """构建整个六足机器人模型（包括身体 + 六条腿）"""
    scene = trimesh.Scene()

    # 加入机器人身体（若有）
    if 'body.stl' in meshes:
        body = meshes['body.stl'].copy()
        body.visual.face_colors = [100, 100, 100, 200]
        scene.add_geometry(body)
    
    # 加入6条腿
    for i in range(1, 7):
        # 腿部可视化
        exp1 = expm(hat(B[(i, 1)]) * alpha[(i, 1)])
        exp2 = expm(hat(B[(i, 2)]) * alpha[(i, 2)])
        exp3 = expm(hat(B[(i, 3)]) * alpha[(i, 3)])
    
        T1 = exp1 @ T[('b', f'b_{i}')] @ T[(f'b_{i}', f'l_{i}_1')]
        T2 = exp1 @ exp2 @ T[('b', f'b_{i}')] @ T[(f'b_{i}', f'l_{i}_1')] @ T[(f'l_{i}_1', f'l_{i}_2')] 
        T3 = exp1 @ exp2 @ exp3 @ T[('b', f'b_{i}')] @ T[(f'b_{i}', f'l_{i}_1')] @ T[(f'l_{i}_1', f'l_{i}_2')] @ T[(f'l_{i}_2', f'l_{i}_3')]
        Te = exp1 @ exp2 @ exp3 @ T[('b', f'b_{i}')] @ T[(f'b_{i}', f'l_{i}_1')] @ T[(f'l_{i}_1', f'l_{i}_2')] @ T[(f'l_{i}_2', f'l_{i}_3')] @ T[(f'l_{i}_3', f'e_{i}')]

        link1 = meshes['link1.stl'].copy()
        link2 = meshes['link2.stl'].copy()
        link3 = meshes['link3.stl'].copy()

        link1.apply_transform(T1)
        link2.apply_transform(T2)
        link3.apply_transform(T3)

        link1.visual.face_colors = [0, 150, 255, 180]
        link2.visual.face_colors = [0, 255, 100, 180]
        link3.visual.face_colors = [255, 0, 0, 180]

        scene.add_geometry(link1)
        scene.add_geometry(link2)
        scene.add_geometry(link3)
        
        # ==== 末端坐标系可视化 ====
        ee_axis = make_axis_mesh()      # 生成一次坐标系
        axis_vis = ee_axis.copy()
        axis_vis.apply_transform(Te)    # 变换到末端位姿
        scene.add_geometry(axis_vis)

    return scene

# === 载入模型文件 ===
model_dir = 'model'
filenames = ['link1.stl', 'link2.stl', 'link3.stl', 'body.stl']
meshes = {}
for fname in filenames:
    path = os.path.join(model_dir, fname)
    if os.path.exists(path):
        meshes[fname] = trimesh.load(path, force='mesh')

# === 构建所有 ===
T_static = build_T_static()
B_axes = compute_screw_axes(T_static)


# 可以设定每一个关节的角度，来查看机器人的形态
# (i, j)第一个数字i代表腿的序号，第二个数字j代表关节的序号
alpha_custom = {
    (1, 1): 0.2, (1, 2): -0.8, (1, 3): -1.0,
    (2, 1): 0.2, (2, 2): -0.2, (2, 3): 0.2,
    (3, 1): 0.2, (3, 2): -0.8, (3, 3): -1.0,
    (4, 1): -0.2, (4, 2): -0.2, (4, 3): 0.2,
    (5, 1): -0.2, (5, 2): -0.8, (5, 3): -1.0,
    (6, 1): -0.2, (6, 2): -0.6, (6, 3): 0.2
}

# === 显示整机结构 ===
scene = build_hexapod(alpha_custom, T_static, B_axes, meshes)



# 显示场景
scene.show()


In [21]:
import numpy as np
from trimesh.scene.cameras import Camera

def set_camera_view(scene, distance=800.0, azimuth=45.0, elevation=30.0, fov=60.0):
    """
    兼容性更好的相机视角设置函数
    参数:
        distance: 相机到目标的距离
        azimuth: 方位角（度）
        elevation: 仰角（度）
        fov: 视野角度（度）
    """
    # 将角度转为弧度
    azimuth_rad = np.radians(azimuth)
    elevation_rad = np.radians(elevation)
    
    # 计算相机位置（球坐标系转笛卡尔坐标）
    target = scene.centroid
    x = target[0] + distance * np.cos(elevation_rad) * np.cos(azimuth_rad)
    y = target[1] + distance * np.cos(elevation_rad) * np.sin(azimuth_rad)
    z = target[2] + distance * np.sin(elevation_rad)
    
    # 创建视图矩阵（手动计算）
    # 相机看向的方向向量
    forward = target - np.array([x, y, z])
    forward /= np.linalg.norm(forward)
    
    # 计算右向量（假设上向量为Z轴）
    up = np.array([0, 0, 1])
    right = np.cross(up, forward)
    right /= np.linalg.norm(right)
    
    # 重新计算真正的上向量
    up = np.cross(forward, right)
    
    # 构建4x4相机变换矩阵
    camera_transform = np.eye(4)
    camera_transform[:3, 0] = right    # X轴（右）
    camera_transform[:3, 1] = up       # Y轴（上）
    camera_transform[:3, 2] = -forward # Z轴（前）
    camera_transform[:3, 3] = [x, y, z] # 位置
    
    # 创建相机对象并设置参数
    camera = Camera(
        fov=(fov, fov),  # 水平和垂直FOV
        resolution=(1920, 1080),
        z_near=0.1,
        z_far=10000
    )
    
    # 应用变换
    scene.camera = camera
    scene.camera_transform = camera_transform

# 使用示例
scene = build_hexapod(alpha_custom, T_static, B_axes, meshes)

# 设置俯视图
set_camera_view(scene, distance=800.0, azimuth=0, elevation=90, fov=60)

# 或者设置等轴测视图
#set_camera_view(scene, distance=800.0, azimuth=45, elevation=30, fov=60)

scene.show()

In [22]:
import trimesh
import numpy as np
import os
from scipy.linalg import expm
import matplotlib.pyplot as plt
from PIL import Image
from trimesh.transformations import rotation_matrix
from tqdm import tqdm


# ========== 以下略去：工具函数和 build_hexapod 保持不变 ==========
# 请确保包含你提供的：_single_arrow, make_axis_mesh, skew, hat, build_hexapod 函数
# ...

# === 创建 output 目录 ===
os.makedirs('output', exist_ok=True)

# === 构建静态部分 ===
T_static = build_T_static()
B_axes   = compute_screw_axes(T_static)

# === 动画参数 ===
n_frames = 30
t_array = np.linspace(0, 4*np.pi, n_frames)
amp1 = 0.4    # 幅度
amp2 = 0.6
fps = 15

# === 渲染并保存每帧 ===
images = []

# tqdm 自动显示循环进度
for t in tqdm(t_array, desc="Rendering frames"):
    alpha_frame = {}
    
    # 草率的步态
    for i in range(1, 7):
        if i==1 or i==3:
            alpha_frame[(i, 1)] = amp1 * np.sin(t)
            alpha_frame[(i, 2)] = amp2 * np.sin(t + np.pi / 2)
            alpha_frame[(i, 3)] = 0.0
        elif i==5:
            alpha_frame[(i, 1)] = -amp1 * np.sin(t)
            alpha_frame[(i, 2)] = amp2 * np.sin(t + np.pi / 2)
            alpha_frame[(i, 3)] = 0.0
        elif i==2:
            alpha_frame[(i, 1)] = -amp1 * np.sin(t)
            alpha_frame[(i, 2)] = -amp2 * np.sin(t + np.pi / 2)
            alpha_frame[(i, 3)] = 0.0
        else:
            alpha_frame[(i, 1)] = amp1 * np.sin(t)
            alpha_frame[(i, 2)] = -amp2 * np.sin(t + np.pi / 2)
            alpha_frame[(i, 3)] = 0.0
            
    scene = build_hexapod(alpha_frame, T_static, B_axes, meshes)
    set_camera_view(scene, distance=800.0, azimuth=45, elevation=45, fov=60)

    png = scene.save_image(resolution=(800, 600), visible=True)  # ✅ 静默渲染
    image = Image.open(trimesh.util.wrap_as_stream(png)).convert("RGB")
    images.append(image)

# === 保存为 GIF 动画 ===
gif_path = os.path.join("output", "hexapod_motion.gif")
images[0].save(gif_path,
               save_all=True,
               append_images=images[1:],
               duration=int(1000 / fps),
               loop=0)

print(f"✅ GIF saved to {gif_path}")


Rendering frames: 100%|██████████| 30/30 [00:33<00:00,  1.12s/it]


✅ GIF saved to output\hexapod_motion.gif
